In [ ]:
from navground import sim
import h5py
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial.distance as dist

import iblofunmatch.inter as ibfm
output_dir="output"
os.makedirs(output_dir, exist_ok=True)

First, we see how the position affects the experiment.

In [ ]:
def compute_square_difference(ibfm_bfun):
    blofun_X = ibfm_bfun[0]["block_function_0"]
    blofun_Y = ibfm_bfun[1]["block_function_0"]
    assignment = [blofun_Y.index(i) for i in blofun_X]
    return np.sum((ibfm_bfun[0]["S_barcode_0"][:,1] - ibfm_bfun[0]["S_barcode_0"][:,1][assignment])**2)/ len(blofun_X)

In [ ]:
side_length = 10
differences_list = []
# Create and run experiments for varying position_noise 
safety_margin_list = []
for safety_margin in [0.1*i for i in range(1, 8, 2)]:
    safety_margin_list.append(safety_margin)
    yaml = f"""
    steps: 1000
    time_step: 0.1
    record_safety_violation: true
    record_task_events: true
    record_pose: true
    save_directory: .
    runs: 1
    scenario:
      type: Cross
      side: {side_length}
      agent_margin: 0.2
      add_safety_to_agent_margin: true
      tolerance: 0.6
      position_noise: 0.1
      groups:
        - number: 30
          type: thymio
          control_period: 0.1
          behavior:
            type: HL
            safety_margin: {safety_margin}
          radius: 0.2
          kinematics:
            type: 2WDiff
            max_speed: 1.0
            wheel_axis: 2
          state_estimation:
            type: Bounded
            range: 2.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run()
    data = h5py.File(experiment.path)
    print(data.keys())
    run = data["run_0"]
    ps = run['poses']
    # Compute matchings for ranging values
    differences = []
    for start_time in range(700):
        end_time = start_time + 20
        # Get start and end points 
        X = ps[start_time][:,[0,1]]
        Y = ps[end_time][:,[0,1]]
        # Compute block functions
        Dist_X = dist.squareform(dist.pdist(X))
        Dist_Y = dist.squareform(dist.pdist(Y))
        Dist_Z = np.minimum(Dist_X, Dist_Y)
        subset_indices = list(range(Dist_X.shape[0]))
        ibfm_bfun = [
            ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, subset_indices, output_dir, max_dim=1),
            ibfm.get_IBloFunMatch_subset(Dist_Y, Dist_Z, subset_indices, output_dir, max_dim=1)
        ]
        # add diferences 
        differences.append(compute_square_difference(ibfm_bfun))
    # end for 
    differences_list.append(differences)

In [ ]:
import matplotlib as mpl
viridis = mpl.colormaps["viridis"]

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
for idx, differences in enumerate(differences_list):
    ax.plot(list(range(len(differences))), differences, c=viridis(idx/len(differences_list)), label=f"safety {safety_margin_list[idx]:.1f}")

ax.legend()
plt.savefig("changing_safety_differences.png")

In [ ]:
yaml = f"""
    steps: 1000
    time_step: 0.1
    record_safety_violation: true
    record_task_events: true
    record_pose: true
    save_directory: .
    runs: 1
    scenario:
      type: Cross
      side: 7
      agent_margin: 0.2
      add_safety_to_agent_margin: true
      tolerance: 0.6
      position_noise: 0.1
      groups:
        - number: 30
          type: thymio
          control_period: 0.1
          behavior:
            type: HL
            safety_margin: 0.4
          radius: 0.6
          color: "#e6e6e6"
          kinematics:
            type: 2WDiff
            max_speed: 1.0
            wheel_axis: 2
          state_estimation:
            type: Bounded
            range: 2.0
    """

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx

def linear_map(a, b, cmap):
    c = cmx.ScalarMappable(norm=colors.Normalize(vmin=a, vmax=b), cmap=cmap)
    def f(v):
        r, g, b, _ = c.to_rgba(v)
        return f"#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}"
    return f

In [ ]:
from navground.sim.ui import WebUI

ui = WebUI(port=8002)
await ui.prepare()

In [ ]:
fill_map = linear_map(0.0, 1.0, cmap=cmx.RdYlGn)

def f(entity):
    if isinstance(entity, sim.Agent):
        return {'fill': fill_map(entity.behavior.efficacy)}
    return {}

ui.decorate = f

In [ ]:
import random
# from navground.sim.ui.video import record_video
from navground.sim.ui.video import display_video
scenario = sim.load_scenario(yaml)
world = sim.World()
scenario.init_world(world, seed=random.randint(0, 2**31))
display_video(world, time_step=0.1, duration=30.0, factor=5.0,
              bounds=((-3, -3), (3, 3)), width=600, decorate=f)